# Download

In [1]:
WORKING_ENV = 'LOCAL' # Can be LABS, COLAB or PAPERSPACE
assert WORKING_ENV in ['LABS', 'COLAB', 'LOCAL']

import sys
import os
if WORKING_ENV == 'COLAB':
  from google.colab import drive
  drive.mount('/content/drive/')
  !pip install medmnist
  !pip install torch
  !pip install gputil
  !pip install psutil
  !pip install humanize
  ROOT = "/content/drive/MyDrive/ColabNotebooks/med-contrastive-project/"
  sys.path.append(ROOT + "./supervised/")
  !nvidia-smi
  slurm_id = 0
elif WORKING_ENV == 'LABS':
  ROOT = "/vol/bitbucket/sx119/Contrastive-Medical-Image-Classification/"
  slurm_id = os.environ["SLURM_JOB_ID"]
else:
  ROOT = "/Users/xushitong/Contrastive-Medical-Image-Classification/"
  slurm_id = 0

Mounted at /content/drive/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116952 sha256=072664fa30803de6249c2c2862dadf9ae585f50cc7f51669dfbc9e2dfee22f75
  Stored in directory: /root/.cache/pip/wheels/f7/f1/89/b9ea2bf8f80ec027a88fef1d354b3816b4d3d29530988972f6
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7408 sha256=f8b56562c410344dded14c6adbc0b9251807833c011367813b38a646ed73126d
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821

# Import

In [2]:
import medmnist

import argparse
import builtins
import math
import os
import random
import shutil
import time
import tqdm
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
# import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
# import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [3]:
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)
print("using device: ", dev)


using device:  cuda:0


In [4]:
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))

mem_report()

CPU RAM Free: 12.4 GB
GPU 0 ... Mem Free: 15098MB / 15360MB | Utilization   0%


# Hyperparameters

In [5]:
EPOCH_NUM = 20
BATCH_SIZE = 128
LEARNING_RATE = 0.03
MOMENTUM = 0.9 # momentum of SGD
WEIGHT_DECAY = 1e-4 # weight decay for SGD
PRINT_FREQ = 10

trial_name = f"epoch{EPOCH_NUM}_batch{BATCH_SIZE}_lr{LEARNING_RATE}_momentum{MOMENTUM}_wd{WEIGHT_DECAY}"

print("trial name: " + trial_name)

trial name: epoch20_batch128_lr0.03_momentum0.9_wd0.0001


# Dataset

In [6]:
class SupervisedDataset():
    """Dataset that applies given data augmentation to each image"""

    def __init__(self, path, augmentation):
        self.samples = torch.load(path)
        self.augmentation = transforms.Compose(augmentation)

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        image, label = self.samples[index]
        return self.augmentation(image), label

In [7]:
# Data loading code
# traindir = os.path.join(args.data, 'train')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
augmentation = [
    transforms.RandomResizedCrop(224, scale=(0.2, 1.)),
    transforms.RandomGrayscale(p=0.2),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
]

# # proper dataset loading, by loading pre-splitted data
pretrain_set = SupervisedDataset(ROOT + "/datasets/pretrain_set.data", augmentation=augmentation)
pretrain_val_set = SupervisedDataset(ROOT + "/datasets/pretrain_val_set.data", augmentation=augmentation)
val_dataset = medmnist.PathMNIST("val", download=False, root=ROOT + "/datasets/", 
                                   transform=transforms.Compose(augmentation))

pretrain_loader = torch.utils.data.DataLoader(
    pretrain_set, batch_size=BATCH_SIZE, shuffle=True, 
    pin_memory=True, drop_last=True)
pretrain_val_loader = torch.utils.data.DataLoader(
    pretrain_val_set, batch_size=2 * BATCH_SIZE, shuffle=False, 
    pin_memory=True, drop_last=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=2 * BATCH_SIZE, shuffle=False, 
    pin_memory=True, drop_last=True)

print(f"pretrain size: {len(pretrain_set)}\npretrain validation size: {len(pretrain_val_set)}\nvalidation size, {len(val_dataset)}")


pretrain size: 80996
pretrain validation size: 9000
validation size, 10004


# Train

In [10]:
if WORKING_ENV == 'LABS':
  summary = open(f"{slurm_id}_{trial_name}.txt", "a")
else:
  summary = sys.stdout


# create model
model = models.resnet50()
model.add_module("projection_head", nn.Linear(1000, 9))
model = model.to(device)

criterion = nn.CrossEntropyLoss(reduction="mean")

optimizer = torch.optim.SGD(model.parameters(), LEARNING_RATE, momentum=0.9, weight_decay=1e-4)

def train_func(img, label, train=True):
  # train/evaluate on given data for one mini batch 
  img = img.to(device)
  label = label.to(device)

  label_hat = model(img)
  l = criterion(label_hat, label.squeeze())

  if train:
    optimizer.zero_grad()
    l.backward()
    optimizer.step()

  return l

def validate(loader):
  acc_l = 0
  with torch.no_grad():
    for (img, label) in loader:
      l = train_func(img, label, False)
      acc_l += l

  summary.write(f"validate loss: {acc_l / len(loader)}\n")



In [11]:
for epoch in range(EPOCH_NUM):
  acc_l = 0
  with tqdm.tqdm(pretrain_loader, unit="batch") as tepoch: 
    for i, (img, label) in enumerate(tepoch):
      l = train_func(img, label)
      acc_l += l.item()

      if i % PRINT_FREQ == 0 and i != 0:
        summary.write(f"Epoch {epoch}[{i}]: loss: {l.item()}[{acc_l / (i + 1)}]")
        validate(pretrain_val_loader)

        if not WORKING_ENV == 'LABS':
          tepoch.set_description(f"batch {i}")
          tepoch.set_postfix(loss=l.item())



  2%|▏         | 10/632 [00:21<22:12,  2.14s/batch]

Epoch 0[10]: loss: 5.238584041595459[10.413460384715687]

batch 10:   2%|▏         | 11/632 [01:27<3:43:07, 21.56s/batch, loss=5.24]

validate loss: 5.393039703369141


batch 10:   3%|▎         | 20/632 [01:45<28:47,  2.82s/batch, loss=5.24]

...

validate loss: 4.843076229095459


batch 20:   5%|▍         | 30/632 [03:09<28:14,  2.82s/batch, loss=6.77]

Epoch 0[30]: loss: 5.072376251220703[7.39904849759994]

batch 20:   5%|▍         | 30/632 [03:41<1:14:13,  7.40s/batch, loss=6.77]


KeyboardInterrupt: ignored

In [ ]:
torch.save(model, f"{slurm_id}_{trial_name}.pickle")
mem_report()